Database operations to build the structure for visualization

I will need 4 groups of data:
* topics (aggregator)
* docs
* words
* people

In [1]:
import pandas as pd
import json
import pickle
import re
import math
import gensim
from collections import Counter

In [2]:
outputs = '../outputs/'

In [3]:
file_tables = '../outputs/tables_dict.pkl'
cpdoc_tables = pickle.load(open(file_tables, 'rb'))

tests

In [391]:
file_tables_v2 = '../outputs/tables_dict-version201906.pkl'
cpdoc_tables_v2 = pickle.load(open(file_tables, 'rb'))

In [ ]:
for table in cpdoc_tables:
    print('table:', table, 'length:', len(cpdoc_tables[table]))

tests end

In [203]:
list(cpdoc_tables)

['person_doc ',
 'top_persons',
 'persons',
 'top_countries',
 'docs',
 'top_topics',
 'doc_counts ',
 'countries',
 'top_classifications',
 'topic_doc',
 'topics',
 'country_doc']

# docs and topics

In [204]:
topic_doc = cpdoc_tables['topic_doc']
topic_doc.head()

,doc_id,topic_id,topic_score,date
0,ag_1973.11.20_doc_I-100,5009,0.167579,1974-02-22
1,ag_1973.11.20_doc_I-100,5010,0.040318,1974-02-22
2,ag_1973.11.20_doc_I-100,5040,0.092381,1974-02-22
3,ag_1973.11.20_doc_I-100,5041,0.090368,1974-02-22
4,ag_1973.11.20_doc_I-100,5051,0.029285,1974-02-22


# docs and persons

In [205]:
persons = cpdoc_tables['persons']
persons = persons.rename(index=str, columns={"id": "person_id"})

In [206]:
person_doc = cpdoc_tables['person_doc ']
person_doc = pd.merge(person_doc, persons, on='person_id', how='inner')
person_doc = person_doc.drop(columns=['date', 'source', 'birth_year', 'death_year', 'description'])
person_doc = person_doc.replace('(.*),(.*),.*', r'\1,\2', regex=True)
person_doc = person_doc.replace('(.*), (.*)', r'\2 \1', regex=True)

In [207]:
#remove special characters
person_doc = person_doc.name.apply(pd.Series) \
    .replace('[“”]', '', regex=True) \
    .replace('[áàãâ]', 'a', regex=True) \
    .replace('[óòõô]', 'o', regex=True) \
    .replace('[éèê]', 'e', regex=True) \
    .replace('[íì]', 'i', regex=True) \
    .replace('[úù]', 'u', regex=True) \
    .replace('ç', 'c', regex=True) \
    .merge(person_doc, left_index = True, right_index = True) \
    .drop("name", axis=1) \
    .rename(columns={0: "name"})

In [208]:
person_doc.head()

,name,person_id,doc_id,person_count
0,Dean Gooderham Acheson,100070,ag_1977.07.00_doc_I-4,1
1,John Quincy Adams,100097,be_1974.03.15_doc_XI-2,1
2,Konrad Adenauer,100118,be_1974.03.15_doc_X-21,1
3,Konrad Adenauer,100118,be_1977.06.01_doc_II-28,1
4,Konrad Adenauer,100118,pn_1974.08.15_doc_I-3,1


In [354]:
person_doc_count = pd.DataFrame(person_doc.groupby(['name'])['person_count'].agg('count')) #.apply(sum)
person_doc_count = person_doc_count.reset_index()

# docs and tokens

In [209]:
docs = cpdoc_tables['docs'][['id', 'title', 'pdf', 'body']]
docs = docs.rename(index=str, columns={"id": "doc_id"})
docs['length'] = docs['body'].apply(lambda x: len(x.split()))
docs.head()

,doc_id,title,pdf,body,length
0,ag_1973.11.20_doc_I-1,Documentos felicitando Azeredo da Silveira pel...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\njr, /h ' ""& m$ hb'h'qo\n& la'?\n\n, é %%3...",261
1,ag_1973.11.20_doc_I-100,Documentos felicitando Azeredo da Silveira pel...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"mjm h\n\n. . . ' "" . ww and\n& !\nzczc tea363 ...",173
2,ag_1973.11.20_doc_I-101,Documentos felicitando Azeredo da Silveira pel...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n \n \n\n \n\n \n\n114"":\n\n \n\n ...",72
3,ag_1973.11.20_doc_I-102,Documentos felicitando Azeredo da Silveira pel...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\n"" "", ""m-nmfvww mmm/""ar? &"", wa "" ""***-**""...",131
4,ag_1973.11.20_doc_I-103,Documentos felicitando Azeredo da Silveira pel...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,\n\n \n\nmi*/gm\n%adazw/sõ'\n\n%% && %%%&\n\n...,295


# words and topics

In [210]:
computer = input("home OR fgv OR broken? ") 
if computer == 'home': outer_inputs = 'D:/'
elif computer == 'broken': outer_inputs = 'C:/Users/marcelo.old-note/Documents/'
else: print('computer is not defined')

home OR fgv OR broken? 
computer is not defined


In [211]:
file_lda_model = outer_inputs+'azeredo_files/model_lda_100_rs_00.pkl'
lda_model = pickle.load(open(file_lda_model, 'rb'))

In [212]:
def remove_special_char(text):
    text = re.sub('[áàãâ]', 'a', text)
    text = re.sub('[óòõô]', 'o', text)
    text = re.sub('[éèê]', 'e', text)
    text = re.sub('[íì]', 'i', text)
    text = re.sub('[úù]', 'u', text)
    text = re.sub('ç', 'c', text)
    return text

In [ ]:
#old version without sorting and selecting only 5 main tokens
def retrieve_tokens(text, token_score):
    main_tokens = []
    for token in token_score['tokens']:
        if token in text:
            token = remove_special_char(token) # refatorar (acaba repetindo mesma limpeza muitas vezes). ou talvez seja melhor assim para nao limpar texto inteiro desnecessariamente...
            main_tokens.append(token)
    return main_tokens

# build vis table
Topic: Nuclear Brazil

In [220]:
main_topics = [['International Cooperation for Development', 99], ['Geisel foreign policy: ideas and action', 61], 
               ['Brazilian government and private investment', 39], ['UN system', 63], 
               ['International Economic Relations of Brazil', 56], ['United States of America', 89], 
               ['Latin America and Caribbean', 4], ['Itaipu plant: technical discussions', 49], ['Nuclear Brazil', 7], 
               ['Brazil, Africa and decolonization', 45]]

In [221]:
def build_vis_table(keyword, topic_doc):
    for pair in main_topics:
        if pair[0] == keyword: topic_num = pair[1]    
    if len(str(topic_num)) == 1: topic_id = '500'+str(topic_num)
    else: topic_id = '50'+str(topic_num)
    topic_id = int(topic_id)
        
    topic_tokens = lda_model.print_topics(-1, num_words=20)[topic_num]
    #topic_tokens[1]
    
    tokens = []
    scores = []
    for i in topic_tokens[1].split('+'):
        token = re.sub('.*\*"(.*)".*', r'\1', i)
        score = re.sub(' *(.*)\*.*', r'\1', i)
        score = float(score)
        tokens.append(token)
        scores.append(score)
    token_score_dict = {'tokens': tokens, 'scores': scores}
    
    token_score = pd.DataFrame(token_score_dict)
    #print(token_score.head())
    
    docs['tokens'] = docs['body'].apply(lambda text: retrieve_tokens(text, token_score))
    
    #docs.head()
    
    #filter
    topic_doc = topic_doc.loc[topic_doc['topic_id'] == topic_id].sort_values(by=['topic_score'], ascending=False)
    topic_doc = topic_doc.head(20)
    
    #merge topics and tokens
    topic_doc = pd.merge(topic_doc, docs, on='doc_id', how='inner')
    topic_doc = topic_doc.drop(columns=['date', 'title'])
    
    #merge topics and persons
    
    array = list(topic_doc['doc_id'])
    person_doc_filtered = person_doc.loc[person_doc['doc_id'].isin(array)]
    
    #merge topics and persons
    #name_count = [row['name'] + '; ' + str(row['person_count']) for index, row in person_doc_filtered.iterrows()]
    #person_doc_filtered['name_count'] = name_count
    name = [row['name'] for index, row in person_doc_filtered.iterrows()]
    person_doc_filtered['name'] = name
    
    person_doc_filtered = person_doc_filtered.groupby(['doc_id'])['name'].apply(list)
    person_doc_filtered = pd.DataFrame({'doc_id':person_doc_filtered.index, 'name':person_doc_filtered.values})
    
    topic_doc = pd.merge(topic_doc, person_doc_filtered, on='doc_id', how='outer')
    topic_doc['name'] = topic_doc['name'] #.fillna(' ')
    for row in topic_doc.loc[topic_doc.name.isnull(), 'name'].index:
        topic_doc.at[row, 'name'] = []
    
    return topic_doc

In [222]:
topic_vis_dict = {}

In [224]:
for pair in main_topics:
    keyword = pair[0]
    topic_vis_dict[keyword] = build_vis_table(keyword, topic_doc)

D:\Users\Marcelo\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [225]:
topic_vis_dict.keys()

dict_keys(['International Cooperation for Development', 'Geisel foreign policy: ideas and action', 'Brazilian government and private investment', 'UN system', 'International Economic Relations of Brazil', 'United States of America', 'Latin America and Caribbean', 'Itaipu plant: technical discussions', 'Nuclear Brazil', 'Brazil, Africa and decolonization'])

In [226]:
topic_vis_dict['Nuclear Brazil'].tail()

,doc_id,topic_id,topic_score,pdf,body,length,tokens,name
15,pn_1974.08.15_doc_III-31,5007,0.303900,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,secreto-excâusãvo\n\nsubsídios para as consult...,4549,"[brasil, armas, nuclear, acordo, nucleares]","[James Earl (Jimmy) Carter, Helmut Schmidt, Cy..."
16,pn_1976.12.28_doc_29,5007,0.300885,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"telegrama recebido '\n\naaaa [97034 . é?\no"" '...",845,"[brasil, armas, acordo, tratado, uranio]",[James Earl (Jimmy) Carter]
17,pn_1974.08.15_doc_II-1,5007,0.290109,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\nmmm\n\n""a. . .. , . ..\n\n' o acordo nucl...",1672,"[nuclear, brasil, fins, energia, acordo]",[James Earl (Jimmy) Carter]
18,pn_1976.12.28_doc_16,5007,0.286767,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,antonio la'. aiii-1315230 da sxmrâxm\njáwówa i...,330,"[brasil, combustivel, aiea, acordo, salvaguardas]",[Antonio Azeredo da Silveira]
19,d_1974.04.23_doc_XXI-12,5007,0.285340,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,\n\n \n\nlembrete n? 011\n\npolítica nuclear ...,1590,"[nuclear, brasil, fins, acordo, energia]",[James Earl (Jimmy) Carter]


In [229]:
topic_vis_dict['Itaipu plant: technical discussions'].tail()

,doc_id,topic_id,topic_score,pdf,body,length,tokens,name
15,bp_1977.03.10_doc_V-3,5049,0.471688,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,". ""* aas_ manoa-to\n/ . _ - . . awm)\n\nsala-1...",6339,"[rio, barragem, construcao, itaipu, energia]","[Antonio Azeredo da Silveira, Costa Cavalcanti..."
16,bp_1977.09.13_doc_II-2,5049,0.469907,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,- informação no? 332/76 - estudo sobre a fixaç...,2444,"[cota, itaipu, corpus, energia, operacao]",[]
17,d_1974.03.26_doc_XXXI-44,5049,0.466800,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\n \n\n \n\nwm\n""mg . .\na"" rio âecreto\n...",6152,"[rio, barragem, construcao, itaipu, energia]","[Antonio Azeredo da Silveira, Costa Cavalcanti..."
18,bp_1977.09.13_doc_I-14,5049,0.455080,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"/\n\n \n\n \n\n \n\na\n\n \n\n!\n\n \n\n o:""i...",1690,"[rio, corpus, iguacu, cota, parana]",[]
19,bp_1977.03.10_doc_III-18,5049,0.452564,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\nwma-03.40\njg?\n(é. /, () h\n\n confidenc...",409,"[nivel, cota, itaipu, operacao, corpus]",[]


# pickle file

In [5]:
topics_dict_file = '../outputs/topics_dict.pkl'

pickle saved for general data already saved. To overwrite:
```python
pickle.dump(topic_vis_dict, open(topics_dict_file, 'wb'))
```

In [6]:
topic_vis_dict = pickle.load(open(topics_dict_file, 'rb'))

# Visualization of metadata

In [19]:
topics_metadata = pd.DataFrame(columns=['mean_score', 'mean_length'])
count=0
for key, df in topic_vis_dict.items():
    score_mean = df['topic_score'].mean()
    length_mean = df['length'].mean()
    row = pd.DataFrame([[key, score_mean, length_mean]], columns=['key', 'mean_score', 'mean_length'], index=[count])
    #row = pd.DataFrame([[score_mean, length_mean]], columns=['mean_score', 'mean_length'], index=[key])
    topics_metadata = topics_metadata.append(row)
    count+=1

D:\Users\Marcelo\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [20]:
topics_metadata

,key,mean_length,mean_score
0,International Cooperation for Development,750.70,0.520749
1,Geisel foreign policy: ideas and action,1490.25,0.371362
2,Brazilian government and private investment,412.45,0.458986
3,UN system,4872.10,0.463308
4,International Economic Relations of Brazil,2686.35,0.489968
5,United States of America,228.80,0.354077
6,Latin America and Caribbean,527.60,0.331226
7,Itaipu plant: technical discussions,2697.70,0.541888
8,Nuclear Brazil,3074.60,0.378028
9,"Brazil, Africa and decolonization",2180.60,0.152870


In [21]:
topics_metadata = topics_metadata.transpose()
#pd.DataFrame(topics_metadata)

In [22]:
topics_metadata

,0,1,2,3,4,5,6,7,8,9
key,International Cooperation for Development,Geisel foreign policy: ideas and action,Brazilian government and private investment,UN system,International Economic Relations of Brazil,United States of America,Latin America and Caribbean,Itaipu plant: technical discussions,Nuclear Brazil,"Brazil, Africa and decolonization"
mean_length,750.7,1490.25,412.45,4872.1,2686.35,228.8,527.6,2697.7,3074.6,2180.6
mean_score,0.520749,0.371362,0.458986,0.463308,0.489968,0.354077,0.331226,0.541888,0.378028,0.15287


In [23]:
topics_metadata_json = topics_metadata.to_json(orient='columns')

In [24]:
topics_metadata_json

'{"0":{"key":"International Cooperation for Development","mean_length":750.7,"mean_score":0.52074945},"1":{"key":"Geisel foreign policy: ideas and action","mean_length":1490.25,"mean_score":0.3713615},"2":{"key":"Brazilian government and private investment","mean_length":412.45,"mean_score":0.4589863},"3":{"key":"UN system","mean_length":4872.1,"mean_score":0.4633078},"4":{"key":"International Economic Relations of Brazil","mean_length":2686.35,"mean_score":0.48996795},"5":{"key":"United States of America","mean_length":228.8,"mean_score":0.3540775},"6":{"key":"Latin America and Caribbean","mean_length":527.6,"mean_score":0.331226},"7":{"key":"Itaipu plant: technical discussions","mean_length":2697.7,"mean_score":0.5418882},"8":{"key":"Nuclear Brazil","mean_length":3074.6,"mean_score":0.3780281},"9":{"key":"Brazil, Africa and decolonization","mean_length":2180.6,"mean_score":0.1528704}}'

In [ ]:
json.dump(topics_metadata_json, open('topics_metadata.json', 'w'))

# Visualization of a specific topic

list split  in pandas: https://mikulskibartosz.name/how-to-split-a-list-inside-a-dataframe-cell-into-rows-in-pandas-9849d8ff2401

In [7]:
nuclear_brazil_df = topic_vis_dict['Nuclear Brazil']
nuclear_brazil_df['doc'] = nuclear_brazil_df['doc_id'].apply(lambda text: re.sub('.*(doc.*)', r'\1', text))
cols = nuclear_brazil_df.columns.tolist()
cols = cols[:3] + cols[4:] + cols[3:4]
nuclear_brazil_df = nuclear_brazil_df[cols]
nuclear_brazil_df.tail()

,doc_id,topic_id,topic_score,body,length,tokens,name,doc,pdf
15,pn_1974.08.15_doc_III-31,5007,0.303900,secreto-excâusãvo\n\nsubsídios para as consult...,4549,"[brasil, armas, nuclear, acordo, nucleares]","[James Earl (Jimmy) Carter, Helmut Schmidt, Cy...",doc_III-31,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...
16,pn_1976.12.28_doc_29,5007,0.300885,"telegrama recebido '\n\naaaa [97034 . é?\no"" '...",845,"[brasil, armas, acordo, tratado, uranio]",[James Earl (Jimmy) Carter],doc_29,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...
17,pn_1974.08.15_doc_II-1,5007,0.290109,"\n\nmmm\n\n""a. . .. , . ..\n\n' o acordo nucl...",1672,"[nuclear, brasil, fins, energia, acordo]",[James Earl (Jimmy) Carter],doc_II-1,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...
18,pn_1976.12.28_doc_16,5007,0.286767,antonio la'. aiii-1315230 da sxmrâxm\njáwówa i...,330,"[brasil, combustivel, aiea, acordo, salvaguardas]",[Antonio Azeredo da Silveira],doc_16,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...
19,d_1974.04.23_doc_XXI-12,5007,0.285340,\n\n \n\nlembrete n? 011\n\npolítica nuclear ...,1590,"[nuclear, brasil, fins, acordo, energia]",[James Earl (Jimmy) Carter],doc_XXI-12,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...


format 1

format 2

format 3

In [8]:
topic1 = nuclear_brazil_df \
    .drop(["body", "topic_id"], axis = 1) \
    .rename(columns={"pdf": "url"})
topic1 = topic1.transpose()
topic1.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
length,7465,3766,1915,3909,4839,1047,1090,10688,9392,384,6106,286,210,1158,251,4549,845,1672,330,1590
tokens,"[armas, nucleares, nuclear, brasil, tratado]","[brasil, armas, nucleares, nuclear, energia]","[armas, tratado, brasil, nucleares, nuclear]","[brasil, nuclear, nucleares, energia, armas]","[nuclear, brasil, armas, nucleares, energia]","[armas, nucleares, brasil, proliferacao, tratado]","[armas, tratado, brasil, salvaguardas, nucleares]","[nuclear, armas, brasil, nucleares, acordo]","[nuclear, brasil, armas, nucleares, energia]","[armas, brasil, nucleares, tecnologia, fins]","[nuclear, armas, brasil, nucleares, tratado]","[nuclear, tecnologia, brasil, alemanha, prolif...","[tecnologia, programa, cooperacao, uranio, com...","[brasil, nuclear, energia, fins, tecnologia]","[nucleares, tratado, armas, nuclear]","[brasil, armas, nuclear, acordo, nucleares]","[brasil, armas, acordo, tratado, uranio]","[nuclear, brasil, fins, energia, acordo]","[brasil, combustivel, aiea, acordo, salvaguardas]","[nuclear, brasil, fins, acordo, energia]"
name,[],[],[],[],[],[],[],"[James Earl (Jimmy) Carter, John Hugh Crimmins]",[],[],[],[James Earl (Jimmy) Carter],[],[Antonio Azeredo da Silveira],[],"[James Earl (Jimmy) Carter, Helmut Schmidt, Cy...",[James Earl (Jimmy) Carter],[James Earl (Jimmy) Carter],[Antonio Azeredo da Silveira],[James Earl (Jimmy) Carter]
doc,doc_4,doc_5,doc_II-11,doc_6,doc_II-1,doc_II-27,doc_II-8,doc_XXIII-31,doc_8,doc_II-22,doc_7,doc_I-23,doc_II-2,doc_XXII-25,doc_6,doc_III-31,doc_29,doc_II-1,doc_16,doc_XXI-12
url,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...


### save json file

In [9]:
topic1_json = topic1.to_json(orient='columns')
topic1_json = re.sub('"\d+":', '', topic1_json)
topic1_json = re.sub("^{", "[", topic1_json)
topic1_json = re.sub("}$", "]", topic1_json)
topic1_json = json.loads(topic1_json)
topic1_json[0]

{'doc_id': 'pn_1975.04.25_doc_4',
 'topic_score': 0.545832,
 'length': 7465,
 'tokens': ['armas', 'nucleares', 'nuclear', 'brasil', 'tratado'],
 'name': [],
 'doc': 'doc_4',
 'url': 'http://www.fgv.br/cpdoc/acervo/arquivo-pessoal/AAS/textual/documentos-referentes-ao-programa-nuclear-brasileiro-durante-a-gestao-de-azeredo-da-silveira-como-ministro-das-relacoes-exteriores-tratando-de-div'}

In [10]:
file_path = outputs+'topic1.json'
json.dump(topic1_json, open(file_path, 'w'))

# Source-target on docs and persons

In [5]:
nuclear_brazil_df = topic_vis_dict['Nuclear Brazil']
nuclear_brazil_df.tail()

,doc_id,topic_id,topic_score,pdf,body,length,tokens,name
15,pn_1974.08.15_doc_III-31,5007,0.303900,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,secreto-excâusãvo\n\nsubsídios para as consult...,4549,"[brasil, armas, nuclear, acordo, nucleares]","[James Earl (Jimmy) Carter, Helmut Schmidt, Cy..."
16,pn_1976.12.28_doc_29,5007,0.300885,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"telegrama recebido '\n\naaaa [97034 . é?\no"" '...",845,"[brasil, armas, acordo, tratado, uranio]",[James Earl (Jimmy) Carter]
17,pn_1974.08.15_doc_II-1,5007,0.290109,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\nmmm\n\n""a. . .. , . ..\n\n' o acordo nucl...",1672,"[nuclear, brasil, fins, energia, acordo]",[James Earl (Jimmy) Carter]
18,pn_1976.12.28_doc_16,5007,0.286767,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,antonio la'. aiii-1315230 da sxmrâxm\njáwówa i...,330,"[brasil, combustivel, aiea, acordo, salvaguardas]",[Antonio Azeredo da Silveira]
19,d_1974.04.23_doc_XXI-12,5007,0.285340,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,\n\n \n\nlembrete n? 011\n\npolítica nuclear ...,1590,"[nuclear, brasil, fins, acordo, energia]",[James Earl (Jimmy) Carter]


format 1

In [7]:
#if you want to build a non relational data. This code repeats doc_id to correspond to multiple names
#.replace(';.*', '', regex=True) \ # if used ';' to quantify name occurrence
topic1 = nuclear_brazil_df.name.apply(pd.Series) \
    .merge(nuclear_brazil_df, left_index = True, right_index = True) \
    .drop(["name", "body", "tokens", "topic_id", 'topic_score', 'length', 'pdf'], axis = 1) \
    .melt(id_vars = ['doc_id'], value_name = "name") \
    .drop("variable", axis = 1) \
    .dropna()

In [8]:
topic1

,doc_id,name
7,d_1974.03.26_doc_XXIII-31,James Earl (Jimmy) Carter
11,pn_1974.08.15_doc_I-23,James Earl (Jimmy) Carter
13,d_1974.03.26_doc_XXII-25,Antonio Azeredo da Silveira
15,pn_1974.08.15_doc_III-31,James Earl (Jimmy) Carter
16,pn_1976.12.28_doc_29,James Earl (Jimmy) Carter
17,pn_1974.08.15_doc_II-1,James Earl (Jimmy) Carter
18,pn_1976.12.28_doc_16,Antonio Azeredo da Silveira
19,d_1974.04.23_doc_XXI-12,James Earl (Jimmy) Carter
27,d_1974.03.26_doc_XXIII-31,John Hugh Crimmins
35,pn_1974.08.15_doc_III-31,Helmut Schmidt


In [310]:
doc_names_source_targets = []
for index, row in topic1.iterrows():
    temp_dict = {'source': row['doc_id'], 'target': row['name']}
    doc_names_source_targets.append(temp_dict)

In [311]:
doc_names_source_targets

[{'source': 'd_1974.03.26_doc_XXIII-31',
  'target': 'James Earl (Jimmy) Carter'},
 {'source': 'pn_1974.08.15_doc_I-23', 'target': 'James Earl (Jimmy) Carter'},
 {'source': 'd_1974.03.26_doc_XXII-25',
  'target': 'Antonio Azeredo da Silveira'},
 {'source': 'pn_1974.08.15_doc_III-31', 'target': 'James Earl (Jimmy) Carter'},
 {'source': 'pn_1976.12.28_doc_29', 'target': 'James Earl (Jimmy) Carter'},
 {'source': 'pn_1974.08.15_doc_II-1', 'target': 'James Earl (Jimmy) Carter'},
 {'source': 'pn_1976.12.28_doc_16', 'target': 'Antonio Azeredo da Silveira'},
 {'source': 'd_1974.04.23_doc_XXI-12', 'target': 'James Earl (Jimmy) Carter'},
 {'source': 'd_1974.03.26_doc_XXIII-31', 'target': 'John Hugh Crimmins'},
 {'source': 'pn_1974.08.15_doc_III-31', 'target': 'Helmut Schmidt'},
 {'source': 'pn_1974.08.15_doc_III-31', 'target': 'Cyrus Vance'}]

In [107]:
file_path = outputs+'doc_names_source_targets.json'
json.dump(doc_names_source_targets, open(file_path, 'w'))

# persons list

In [313]:
list(topic1.name.unique())

['James Earl (Jimmy) Carter',
 'Antonio Azeredo da Silveira',
 'John Hugh Crimmins',
 'Helmut Schmidt',
 'Cyrus Vance']

In [386]:
names_list = []
for name in list(topic1.name.unique()):
    name_count = math.log(int(person_doc_count.loc[person_doc_count['name'] == name]['person_count']),1.5)
    temp_dict = {'name': name, 'count': name_count}
    names_list.append(temp_dict)

In [389]:
names_list

[{'name': 'James Earl (Jimmy) Carter', 'count': 15.507767337860294},
 {'name': 'Antonio Azeredo da Silveira', 'count': 19.6887445579689},
 {'name': 'John Hugh Crimmins', 'count': 12.94075363634189},
 {'name': 'Helmut Schmidt', 'count': 8.69707517144841},
 {'name': 'Cyrus Vance', 'count': 12.274135660257478}]

In [375]:
math.log(2.7188)

1.000190606490346

In [380]:
math.log(2)

0.6931471805599453

In [381]:
math.log(10)

2.302585092994046

In [388]:
file_path = outputs+'names_list.json'
json.dump(names_list, open(file_path, 'w'))

# Source-target on docs and tokens

In [235]:
nuclear_brazil_df = topic_vis_dict['Nuclear Brazil']
nuclear_brazil_df['doc'] = nuclear_brazil_df['doc_id'].apply(lambda text: re.sub('.*(doc.*)', r'\1', text))
nuclear_brazil_df.tail()

,doc_id,topic_id,topic_score,pdf,body,length,tokens,name,doc
15,pn_1974.08.15_doc_III-31,5007,0.303900,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,secreto-excâusãvo\n\nsubsídios para as consult...,4549,"[brasil, armas, nuclear, acordo, nucleares]","[James Earl (Jimmy) Carter, Helmut Schmidt, Cy...",doc_III-31
16,pn_1976.12.28_doc_29,5007,0.300885,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"telegrama recebido '\n\naaaa [97034 . é?\no"" '...",845,"[brasil, armas, acordo, tratado, uranio]",[James Earl (Jimmy) Carter],doc_29
17,pn_1974.08.15_doc_II-1,5007,0.290109,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\nmmm\n\n""a. . .. , . ..\n\n' o acordo nucl...",1672,"[nuclear, brasil, fins, energia, acordo]",[James Earl (Jimmy) Carter],doc_II-1
18,pn_1976.12.28_doc_16,5007,0.286767,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,antonio la'. aiii-1315230 da sxmrâxm\njáwówa i...,330,"[brasil, combustivel, aiea, acordo, salvaguardas]",[Antonio Azeredo da Silveira],doc_16
19,d_1974.04.23_doc_XXI-12,5007,0.285340,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,\n\n \n\nlembrete n? 011\n\npolítica nuclear ...,1590,"[nuclear, brasil, fins, acordo, energia]",[James Earl (Jimmy) Carter],doc_XXI-12


format 1

In [11]:
#if you want to build a non relational data. This code repeats doc_id to correspond to multiple names
#.replace(';.*', '', regex=True) \ # if used ';' to quantify name occurrence
topic1 = nuclear_brazil_df.tokens.apply(pd.Series) \
    .merge(nuclear_brazil_df, left_index = True, right_index = True) \
    .drop(["name", "body", "tokens", "topic_id", 'topic_score', 'length', 'doc', 'pdf'], axis = 1) \
    .melt(id_vars = ['doc_id'], value_name = "tokens") \
    .drop("variable", axis = 1) \
    .dropna()

In [13]:
doc_tokens_source_targets = []
for index, row in topic1.iterrows():
    temp_dict = {'source': row['doc_id'], 'target': row['tokens']}
    doc_tokens_source_targets.append(temp_dict)

In [239]:
file_path = outputs+'doc_tokens_source_targets.json'
json.dump(doc_tokens_source_targets, open(file_path, 'w'))

# tokens list

In [266]:
for pair in main_topics:
    if pair[0] == 'Nuclear Brazil': topic_num = pair[1]
topic_tokens = lda_model.print_topics(-1, num_words=20)[topic_num]
topic_tokens = topic_tokens[1].split('+')
topic_tokens

['0.085*"nuclear" ',
 ' 0.036*"acordo" ',
 ' 0.035*"energia" ',
 ' 0.030*"nucleares" ',
 ' 0.023*"brasil" ',
 ' 0.020*"urânio" ',
 ' 0.020*"armas" ',
 ' 0.017*"tecnologia" ',
 ' 0.016*"rfa" ',
 ' 0.014*"tratado" ',
 ' 0.014*"cooperação" ',
 ' 0.012*"salvaguardas" ',
 ' 0.011*"programa" ',
 ' 0.010*"aiea" ',
 ' 0.009*"combustível" ',
 ' 0.008*"proliferação" ',
 ' 0.007*"reatores" ',
 ' 0.007*"alemanha" ',
 ' 0.007*"atômica" ',
 ' 0.007*"fins"']

In [272]:
tokens_list = []
for i in topic_tokens:
    token = re.sub('.*\*"(.*)".*', r'\1', i)
    token = remove_special_char(token)
    score = re.sub(' *(.*)\*.*', r'\1', i)
    score = float(score)
    
    #remove tokens irrelevantes
    if token == 'fins': continue
        
    temp_dict = {'token': token, 'score': score}
    tokens_list.append(temp_dict)

In [274]:
file_path = outputs+'tokens_list.json'
json.dump(tokens_list, open(file_path, 'w'))